In [1]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import os

# Đường dẫn tới thư mục ảnh đen trắng sau khi đã phân tách
bw_output_dir = Path("Project_test2/picture2")

# Đảm bảo thư mục lưu file CSV tồn tại
csv_output_dir = Path("Project_test2/data2")
csv_output_dir.mkdir(parents=True, exist_ok=True)

# Hàm phân loại diện tích vật thể theo nhóm phần trăm
def classify_area_percentage(area_percentage):
    if area_percentage < 1:
        return "0-1%"  
    elif area_percentage < 10:
        return "1-10%"
    elif area_percentage < 20:
        return "10-20%"
    elif area_percentage < 70:
        return "20-70%"
    else:
        return "70-100%"

# Kiểm tra xem vật thể có dính nền không
def check_object_background(bw_img):
    contours, _ = cv2.findContours(bw_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 0:
            x, y, w, h = cv2.boundingRect(contour)
            edge_pixels = bw_img[y:y+h, x:x+w].flatten()
            if np.any(edge_pixels == 255):
                return "Có thể có dính nền"
    return "Không dính nền"

# Thống kê phần trăm diện tích vật thể cho mỗi ảnh trong thư mục và lưu vào CSV
results = []

for subfolder in bw_output_dir.glob("*"):  # Mỗi thư mục con chứa ảnh gốc và ảnh vật thể
    if subfolder.is_dir():
        original_img_path = subfolder / "original.png"
        objects = list(subfolder.glob("object_*_bw.png"))
        
        for obj_path in objects:
            # Đọc ảnh đen trắng
            bw_img = cv2.imread(str(obj_path), 0)
            if bw_img is None:
                continue  # Bỏ qua nếu ảnh không tải được
            
            h, w = bw_img.shape
            area_percentage = (np.sum(bw_img == 255) / (h * w)) * 100
            area_category = classify_area_percentage(area_percentage)
            background_check = check_object_background(bw_img)
            
            # Lưu kết quả cho mỗi vật thể
            results.append({
                "image_name": subfolder.name,
                "object_name": obj_path.stem,
                "area_percentage": area_percentage,
                "area_category": area_category,
                "background_check": background_check
            })

# Lưu kết quả thống kê vào file CSV
df_results = pd.DataFrame(results)
csv_output_path = csv_output_dir / "object_analysis2.csv"
df_results.to_csv(csv_output_path, index=False)

# Báo cáo tổng quan về diện tích vật thể
print("Thống kê diện tích vật thể đã tách:")
print(df_results["area_category"].value_counts())

print(f"\nKết quả đã lưu thành công vào {csv_output_path}")


Thống kê diện tích vật thể đã tách:
area_category
0-1%       48670
1-10%      25632
10-20%      4964
20-70%      4263
70-100%      479
Name: count, dtype: int64

Kết quả đã lưu thành công vào Project_test2\data2\object_analysis2.csv
